In [118]:
import adip
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
import geopandas as gpd
from constants import CRS
import nhgis

In [101]:
airports = adip.get_adip()

# will fix underlying soon
airports = airports.groupby('Loc Id').head(1)
airports = airports.set_index('Loc Id')
airports = airports[['Name', 'geometry']]

In [37]:
# get the coordinates
coords = list(zip(airports['geometry'].x, airports['geometry'].y))

# add a bounding box
coords.append((-180, 90))
coords.append((180, 90))
coords.append((-180, -90))
coords.append((180, -90))

In [42]:
# run SciPy voronoi code
vor = Voronoi(coords)

In [45]:
# get regions
r = vor.regions

In [46]:
# replace corners with empty arrays
r = [i if -1 not in i else [] for i in r]
r = [i if len(i) >= 3 else [] for i in r]

In [48]:
# get vertices
v = vor.vertices

In [53]:
# create polygons
p = []
for reg in r:
    p.append(Polygon([v[i] for i in reg]))

In [67]:
# input point to output region
ptr = vor.point_region

In [77]:
# sorted regions
# remove bounding box
sorted = [p[i] for i in ptr][:-4]

In [78]:
len(sorted)

4344

In [95]:
# geoseries of regions
g = gpd.GeoSeries(sorted)
g.crs = CRS
g.name = 'region'

In [102]:
g

0       POLYGON ((-177.15795 59.05234, -176.23801 57.9...
1       POLYGON ((-155.77792 57.29345, -156.10714 55.4...
2       POLYGON ((-161.66688 60.85034, -161.85726 61.3...
3       POLYGON ((-161.34437 61.33183, -161.28317 61.3...
4       POLYGON ((-163.13505 51.88194, -165.06117 57.1...
                              ...                        
4339    POLYGON ((-104.69482 42.59091, -104.88161 43.0...
4340    POLYGON ((-104.36777 42.37659, -103.73789 42.6...
4341    POLYGON ((-105.02936 44.43088, -105.27056 43.5...
4342    POLYGON ((-107.12461 43.55039, -107.35838 44.2...
4343    POLYGON ((-108.55357 44.44843, -108.55408 44.2...
Name: region, Length: 4344, dtype: geometry

In [104]:
df = airports.reset_index().join(g)

In [113]:
airports = df.set_geometry('region')[['Loc Id', 'Name', 'region']]

In [114]:
airports = airports.set_index('Loc Id')

In [116]:
airports.head()

,Name,region
Loc Id,,
ADK,ADAK,"POLYGON ((-177.15795 59.05234, -176.23801 57.9..."
AKK,AKHIOK,"POLYGON ((-155.77792 57.29345, -156.10714 55.4..."
Z13,AKIACHAK,"POLYGON ((-161.66688 60.85034, -161.85726 61.3..."
AKI,AKIAK,"POLYGON ((-161.34437 61.33183, -161.28317 61.3..."
7AK,AKUTAN,"POLYGON ((-163.13505 51.88194, -165.06117 57.1..."


In [130]:
census = gpd.read_file('cache/uscb/merged/tract/tract.shp')

In [165]:
intersections = airports.sjoin(census, how='left', predicate='intersects').reset_index()
intersections = intersections.merge(census[['index', 'geometry']], on='index').set_index('Loc Id')

In [172]:
pton = intersections.loc['39N'].copy()

In [173]:
pton['overlap'] = pton['region'].intersection(pton['geometry'])

In [180]:
pton['ratio'] = pton['overlap'].to_crs('+proj=cea').area / pton['geometry'].to_crs('+proj=cea').area

In [182]:
pton['in_pop'] = pton['POP']*pton['ratio']

In [184]:
pton['in_pop'].sum()

138344.56727398283